# App for running GECCO jobs in Galaxy

1. Upload and run workflow.
2. Monitor the job.
3. Receive completion notification with some basic summary provided by Galaxy.

Note: "Receiving" the results (tentatively download) is part of the analysis pipeline.

In [1]:
import os
import sys
import json
from datetime import datetime
from platform import python_version
import logging

# Import
import bioblend.galaxy as g  # BioBlend is a Python library, wrapping the functionality of Galaxy and CloudMan APIs
# import boto3
import pandas as pd
import panel as pn
# from bioblend.galaxy import GalaxyInstance
# from bioblend.galaxy.config import ConfigClient

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from utils import init_setup
init_setup()

from momics.galaxy import BCGalaxy

# instead of the jupyter magic, you can also use
from dotenv import load_dotenv
load_dotenv()
# %load_ext dotenv
# %dotenv

local Linux


True

In [2]:

FORMAT = "%(asctime)s | %(levelname)s | %(name)s | %(message)s"

@pn.cache
def reconfig_basic_config(format_=FORMAT, level=logging.INFO):
    """(Re-)configure logging"""
    logging.basicConfig(format=format_, level=level, force=True)
    logging.info("Logging.basicConfig completed successfully")

In [4]:
exp = BCGalaxy("GALAXY_EARTH_URL", "GALAXY_EARTH_KEY")

In [7]:
## debugging

# exp.get_histories()
# exp.histories

# exp.set_history(create=False, hid="8d8d4bf21253beda")

## Environment setup

In [38]:
# setup the galaxy environment

In [41]:
if 'google.colab' in str(get_ipython()):
    root_folder = os.path.abspath(os.path.join('/content/momics-demos'))
    # data_folder = os.path.join(root_folder, 'parquet_files')
else:
    root_folder = os.path.abspath(os.path.join('../'))

assets_folder = os.path.join(root_folder, 'assets')

## App setup

In [37]:
import panel as pn

In [9]:
def handle_login(clicks):
    exp = BCGalaxy("GALAXY_EARTH_URL", "GALAXY_EARTH_KEY")
    pn.state.notifications.info(f"User logged in: {exp.cfg.whoami()}")
    logger.info(f'You have clicked me {clicks} times')

In [8]:
# buttons
but_login = pn.widgets.Button(name="🔐 Galaxy Login")

In [ ]:
# histories
get_histories = pn.widgets.Button(name="📚 Get Histories")

In [ ]:
reconfig_basic_config()
logger = logging.getLogger(name="app")

pn.extension("tabulator")
pn.extension(notifications=True)
ACCENT = "teal"

styles = {
    "box-shadow": "rgba(50, 50, 93, 0.25) 0px 6px 12px -2px, rgba(0, 0, 0, 0.3) 0px 3px 7px -3px",
    "border-radius": "4px",
    "padding": "10px",
}

# TODO: there is a bug in the panel library that does not allow to open png files, renoming does not help 
image = pn.pane.JPG(os.path.join(assets_folder, "figs/metaGOflow_logo_italics.jpg"),
                    width=200, height=100)




template = pn.template.FastListTemplate(
    title="Run GECCO on Galaxy",
    sidebar=[image,
             but_login, pn.bind(handle_login, clicks=but_login.param.clicks),
             pn.layout.Divider(),
             
             ],
    main=[pn.Column("## Histories", 
                    # get_histories, pn.bind(handle_login, clicks=but_login.param.clicks), select_history,
                    "## Datasets", #get_datasets, select_dataset,
                    sizing_mode="stretch_both",
                   )],
    main_layout=None,
    accent=ACCENT,
    meta_refresh="2",
)
logger.info("Template created")
template.servable()